In [ ]:
print(0)

---
# MelanoVision

---

Credit to pedrocast7 for the EDA starter code

In [2]:
import numpy as np # linear algebra
import numpy.matlib
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
from PIL import Image
import cv2
import random
import os ## handle files
import torch ## PyTorch
import torch.nn as nn ## Neural networks package
from torch import optim ## optimizer
import torch.optim.lr_scheduler as lr_scheduler
from torchvision import models ## package consists of popular datasets, model architectures, and common image transformations for computer vision.
from torch.utils.data import Dataset, DataLoader, ConcatDataset, TensorDataset
import torchvision.transforms as transforms ## for data augmentation
from sklearn.model_selection import train_test_split ## to split datasets
import seaborn as sns ## data plot
from sklearn.metrics import confusion_matrix ## plot the confusion matrix

In [ ]:
# Setting the random seed for reproducibility
seed = 77
np.random.seed(seed) ## for numpy
torch.manual_seed(seed) ## for PyTorch
torch.cuda.manual_seed(seed)
random.seed(24) ## for random module on python


# Check if a GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("GPU is not available.")
    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") ## use gpu if its available

In [ ]:
## Path to data
HAM1000_path = r'C:\Users\ege\git\MelanoVision\ham1'

## to map the class acronym to its real name, according to the paper of the dataset
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

## collecting image paths
all_image_path = glob(os.path.join(HAM1000_path, '*', '*.jpg'))

## imageid_path_dict will map each image ID (the file name without the extension) to its full file path.
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}


df_original = pd.read_csv(os.path.join(HAM1000_path, 'HAM10000_metadata.csv'))
df_original['path'] = df_original['image_id'].map(imageid_path_dict.get)
df_original['cell_type'] = df_original['dx'].map(lesion_type_dict.get)
df_original['cell_type_idx'] = pd.Categorical(df_original['cell_type']).codes

## class names
classes = df_original['dx'].unique()

df_original.head()

In [ ]:
# Count the number of images in each class
class_counts = df_original['cell_type'].value_counts()

# Plot a histogram
plt.figure(figsize=(10, 6))
class_counts.plot(kind='barh')

# Add title and labels
plt.title('Number of Images per Class')
plt.ylabel('Class')
plt.xlabel('Number of Images')

# Display the plot
plt.tight_layout()
plt.show()


print(class_counts)

In [ ]:
%%time
## Showing some examples of the data

# Randomly select 15 rows from the dataframe
sample_df = df_original.sample(n=15)

# Set up the matplotlib figure and axes
fig, axes = plt.subplots(3, 5, figsize=(20, 12))
axes = axes.flatten()

for ax, (_, row) in zip(axes, sample_df.iterrows()):
    img_path = row['path']
    cell_type = row['cell_type']
    
    # Open the image
    img = Image.open(img_path)
    
    # Display the image on the axis
    ax.imshow(img)
    ax.set_title(cell_type)
    ax.axis('off')  # Hide the axes ticks

# Add a main title for the whole plot
plt.suptitle('HAM10000 Examples', fontsize=20)
# Adjust the layout
plt.tight_layout()
plt.show()


In [8]:
## Plots the Train/Val curves (Losses and Main Metric) 
def plot_history(train_loss_hist, val_loss_hist, train_acc_hist, val_acc_hist, model_name):
    epochs = range(1, len(train_loss_hist) + 1)
    
    # Plotting the loss history
    plt.figure(figsize=(12, 5))
    
    # Loss Plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss_hist, 'bo-', label='Training Loss')
    plt.plot(epochs, val_loss_hist, 'ro-', label='Validation Loss')
    plt.title(f'Training and Validation Loss: {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid()
    plt.legend()
    
    # Accuracy Plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc_hist, 'bo-', label='Training Accuracy')
    plt.plot(epochs, val_acc_hist, 'ro-', label='Validation Accuracy')
    plt.title(f'Training and Validation Balanced Accuracy: {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('Balanced Accuracy')
    plt.ylim(0.4, 1)  # Set y-axis limit from 0.4 to 1
    plt.grid()
    plt.legend()
    
    plt.tight_layout()
    plt.show()

    
## Plots the confusion matrix given a prediction and target labels
def plot_confusion_matrix(all_predictions, all_labels, num_classes, model_name):
    '''Plot the confusion matrix using predictions and true labels'''
    cm = confusion_matrix(all_labels.cpu().numpy(), all_predictions.cpu().numpy())
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='inferno', xticklabels=range(num_classes), yticklabels=range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix: {model_name}')
    plt.show()

    
## gets the mean and std from a given dataset
def compute_img_mean_std(image_paths):
    """
        computing the mean and std of three channel on the whole dataset,
        first we should normalize the image from 0-255 to 0-1
    """
    
    img_h, img_w = 120, 120
    imgs = []
    means, stdevs = [], []

    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        imgs.append(img)

    imgs = np.stack(imgs, axis=3)
    print(imgs.shape)

    imgs = imgs.astype(np.float32) / 255.

    for i in range(3):
        pixels = imgs[:, :, i, :].ravel()  # resize to one row
        means.append(np.mean(pixels))
        stdevs.append(np.std(pixels))

    means.reverse()  # BGR --> RGB
    stdevs.reverse()

    print("normMean = {}".format(means))
    print("normStd = {}".format(stdevs))
    return means,stdevs

## Define the custom dataset class
class CustomHAM10000(Dataset):
    def __init__(self, dataframe, img_size, transform=None):
        self.paths = dataframe['path'].values
        self.labels = torch.tensor(dataframe['cell_type_idx'].values, dtype=torch.long)
        self.transform = transform
        self.img_size = img_size

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        image = Image.open(img_path)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
import os
import glob
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image
import timm  # Library to provide transformer models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [10]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Use train_test_split to split the dataset into train and test
train_df, test_df = train_test_split(df_original, test_size=0.2, random_state=42, stratify=df_original['cell_type_idx'])

# Create dataset instances
train_dataset = CustomHAM10000(train_df, img_size=(224, 224), transform=transform)
test_dataset = CustomHAM10000(test_df, img_size=(224, 224), transform=transform)

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
"""
model = timm.create_model('vit_base_patch16_224', pretrained=True).to(device)
model.head = nn.Linear(model.head.in_features, len(classes))

optimizer = optim.Adam(model.parameters(), lr=0.0001)
"""


criterion = nn.CrossEntropyLoss()

In [ ]:
# Training function
def train_epoch(model, data_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    correct_predictions = 0
    for inputs, labels in tqdm(data_loader, desc='Training'):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_predictions += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / len(data_loader.dataset)
    epoch_acc = correct_predictions.double() / len(data_loader.dataset)
    
    return epoch_loss, epoch_acc

# Evaluation function
def evaluate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0
    correct_predictions = 0
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc='Evaluating'):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(data_loader.dataset)
    epoch_acc = correct_predictions.double() / len(data_loader.dataset)

    return epoch_loss, epoch_acc


In [13]:
model1 = timm.create_model('vit_base_patch16_224', pretrained=True).to(device)
model1.head = nn.Linear(model1.head.in_features, len(classes))  # Adjust the final layer to match number of classes
optimizer1 = optim.Adam(model1.parameters(), lr=0.0001)

for k, v in model1.named_parameters():
    v.requires_grad = True

In [ ]:
from torchsummary import summary
#summary(model1, input_size=(3, 224, 224))
print(model1)

In [ ]:
def model_details(model):
    print("Layer Name".ljust(30), "Output Shape".ljust(30), "Param #")
    print("="*60)
    total_params = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            layer_params = param.numel()
            total_params += layer_params
            print(f"{name}".ljust(30), str(param.shape).ljust(30), layer_params)
    print("="*60)
    print(f"Total Parameters: {total_params}")

model_details(model1)

In [ ]:
for k, v in model1.named_parameters():
    print(k)
    #print(v)

In [ ]:
modelx = timm.create_model('vit_base_patch16_224', pretrained=True).to(device)

optimizerx = optim.AdamW(filter(lambda p: p.requires_grad, modelx.parameters()), lr=0.0001)

trainable = sum([v.numel() for _,v in model1.named_parameters() if v.requires_grad])

print(f"Trainable Parameters: {trainable:,}")
num_epochs = 3

for epoch in range(num_epochs):
    print('-' * 15)
    print('-' * 15)
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('-' * 10)

    train_loss, train_acc = train_epoch(modelx, train_loader, criterion, optimizerx, device)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    
    test_loss, test_acc = evaluate(modelx, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

In [ ]:
trainable = sum([v.numel() for _,v in model1.named_parameters() if v.requires_grad])

print(f"Trainable Parameters: {trainable:,}")
num_epochs = 3

for epoch in range(num_epochs):
    print('-' * 15)
    print('-' * 15)
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('-' * 10)

    train_loss, train_acc = train_epoch(model1, train_loader, criterion, optimizer1, device)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    
    test_loss, test_acc = evaluate(model1, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

In [ ]:
model2 = timm.create_model('vit_base_patch16_224', pretrained=True).to(device)
model2.head = nn.Linear(model2.head.in_features, len(classes))  # Adjust the final layer to match number of classes
optimizer2 = optim.Adam(model1.parameters(), lr=0.0001)

for k, v in model2.named_parameters():
    #print(k, str(v.requires_grad)[0], "-> ", end=''   )#, v)
    if (k[:7]=="blocks."):
        v.requires_grad = ("bias" in k)
    #print(str(v.requires_grad)[0])
    #v.requires_grad = v.requires_grad


In [ ]:
trainable = sum([v.numel() for _,v in model2.named_parameters() if v.requires_grad])

print(f"Trainable Parameters: {trainable:,}")

num_epochs = 3

for epoch in range(num_epochs):
    print('-' * 15)
    print('-' * 15)
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('-' * 10)

    train_loss, train_acc = train_epoch(model2, train_loader, criterion, optimizer2, device)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    
    test_loss, test_acc = evaluate(model2, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')


In [16]:
model3 = timm.create_model('vit_base_patch16_224', pretrained=True).to(device)
model3.head = nn.Linear(model3.head.in_features, len(classes))  # Adjust the final layer to match number of classes
optimizer3 = optim.Adam(model3.parameters(), lr=0.0001)

for k, v in model3.named_parameters():
    #print(k, str(v.requires_grad)[0], "-> ", end=''   )#, v)
    if (k[:7]=="blocks."):
        v.requires_grad = ("bias" in k)
    #print(str(v.requires_grad)[0])
    #v.requires_grad = v.requires_grad

In [ ]:
trainable = sum([v.numel() for _,v in model3.named_parameters() if v.requires_grad])

print(f"Trainable Parameters: {trainable:,}")

num_epochs = 10

for epoch in range(num_epochs):
    print('-' * 15)
    print('-' * 15)
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('-' * 10)

    train_loss, train_acc = train_epoch(model3, train_loader, criterion, optimizer3, device)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    
    test_loss, test_acc = evaluate(model3, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')